**Getting necessary libraries**

In [1]:
import requests 
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import re
import random

In [2]:
import sys
print(sys.version)


3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:17:24) [GCC 13.3.0]


**Against Permissions accessing**

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
    "Connection": "keep-alive",
    "DNT": "1",  # Do Not Track request
    "Upgrade-Insecure-Requests": "1",
    "Cache-Control": "no-cache",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1"
}

**Important Links**
- Points Table Page: https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/points-table-standings
- All IPL Seasons Page: https://www.espncricinfo.com/records/trophy/team-match-results-season/indian-premier-league-117
- IPL T20 Website: https://www.iplt20.com/teams
- Complete Aution players: https://www.espncricinfo.com/auction/ipl-2025-auction-1460972

## 0: Teams Information

In [4]:
# dim_teams: `team_code`, `team`, `team_logo`, `captain`, `captain_img`

# -----------------> 2025 Data ------------------------->


# 1: short codes
ipl_teams_names = {
    "CSK": "Chennai Super Kings",
    "DC": "Delhi Capitals",
    "GT": "Gujarat Titans",
    "KKR": "Kolkata Knight Riders",
    "LSG": "Lucknow Super Giants",
    "MI": "Mumbai Indians",
    "PBKS": "Punjab Kings",
    "RR": "Rajasthan Royals",
    "RCB": "Royal Challengers Bengaluru",
    "SRH": "Sunrisers Hyderabad"
}




# 2: team logos
ipl_teams_logos = {
    'KKR': 'https://documents.iplt20.com/ipl/KKR/Logos/Logooutline/KKRoutline.png',
    'SRH': 'https://documents.iplt20.com/ipl/SRH/Logos/Logooutline/SRHoutline.png',
    'RR': 'https://documents.iplt20.com/ipl/RR/Logos/Logooutline/RRoutline.png',
    'RCB': 'https://documents.iplt20.com/ipl/RCB/Logos/Logooutline/RCBoutline.png',
    'CSK': 'https://documents.iplt20.com/ipl/CSK/logos/Logooutline/CSKoutline.png',
    'DC': 'https://documents.iplt20.com/ipl/DC/Logos/LogoOutline/DCoutline.png',
    'LSG': 'https://documents.iplt20.com/ipl/LSG/Logos/Logooutline/LSGoutline.png',
    'GT': 'https://documents.iplt20.com/ipl/GT/Logos/Logooutline/GToutline.png',
    'PBKS': 'https://documents.iplt20.com/ipl/PBKS/Logos/Logooutline/PBKSoutline.png', 
    'MI': 'https://documents.iplt20.com/ipl/MI/Logos/Logooutline/MIoutline.png'
}




# create dataframe
df_ipl_teams = pd.DataFrame(list(ipl_teams_names.items()), columns=["team_code", "team"])

# add captain name, logo and other info
df_ipl_teams['team_logo'] = df_ipl_teams['team_code'].map(ipl_teams_logos)

print(df_ipl_teams.shape)
df_ipl_teams.head(10)

(10, 3)


,team_code,team,team_logo
0,CSK,Chennai Super Kings,https://documents.iplt20.com/ipl/CSK/logos/Log...
1,DC,Delhi Capitals,https://documents.iplt20.com/ipl/DC/Logos/Logo...
2,GT,Gujarat Titans,https://documents.iplt20.com/ipl/GT/Logos/Logo...
3,KKR,Kolkata Knight Riders,https://documents.iplt20.com/ipl/KKR/Logos/Log...
4,LSG,Lucknow Super Giants,https://documents.iplt20.com/ipl/LSG/Logos/Log...
5,MI,Mumbai Indians,https://documents.iplt20.com/ipl/MI/Logos/Logo...
6,PBKS,Punjab Kings,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...
7,RR,Rajasthan Royals,https://documents.iplt20.com/ipl/RR/Logos/Logo...
8,RCB,Royal Challengers Bengaluru,https://documents.iplt20.com/ipl/RCB/Logos/Log...
9,SRH,Sunrisers Hyderabad,https://documents.iplt20.com/ipl/SRH/Logos/Log...


## 1: Points Table

In [5]:
# Step 1: Send a GET request to the website
url = 'https://www.espncricinfo.com/series/ipl-2025-1449924/points-table-standings'
response = requests.get(url, headers=headers)

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# step3: Get the table
tables = soup.select('div > table.ds-table')[0]

# step4: Iterate through each row
allRows = soup.select('table > tbody > tr.ds-text-tight-s')
pointsTable = []

for element in allRows:
    tds = element.find_all('td')
     
    team = re.sub(r'\d+', '', tds[0].text.strip())
    total_matches_played = tds[1].text.strip()
    total_matches_won = tds[2].text.strip()
    total_matches_loss = tds[3].text.strip()
    total_matches_tied = tds[4].text.strip()
    total_matches_abonded = tds[5].text.strip()
    points_awarded = tds[6].text.strip()
    net_run_rate = tds[7].text.strip()
    
    pointsTable.append({
        'team': team,
        'matches_played': total_matches_played,
        'won': total_matches_won,
        'lost': total_matches_loss,
        'tied': total_matches_tied,
        'abonded': total_matches_abonded,
        'points': points_awarded,
        "net_run_rate": net_run_rate
    })


# Step 5: Convert the data into a pandas DataFrame
df_points_table = pd.DataFrame(pointsTable)
# let's add short code of team
ipl_teams = {
    'Kolkata Knight Riders': 'KKR',
    'Sunrisers Hyderabad': 'SRH',
    'Rajasthan Royals': 'RR',
    'Royal Challengers Bengaluru': 'RCB',
    'Chennai Super Kings': 'CSK',
    'Delhi Capitals': 'DC',
    'Lucknow Super Giants': 'LSG',
    'Gujarat Titans': 'GT',
    'Punjab Kings': 'PBKS',
    'Mumbai Indians': 'MI'
}
df_points_table['team_code'] = df_points_table['team'].map(ipl_teams)

#re-arrange
cols = ['team_code', 'team', 'matches_played', 'won', 'lost', 'tied', 'abonded', 'points', 'net_run_rate']
df_points_table = df_points_table[cols]

print(df_points_table.shape)
df_points_table.head(10)


(10, 9)


,team_code,team,matches_played,won,lost,tied,abonded,points,net_run_rate
0,RCB,Royal Challengers Bengaluru,2,2,0,0,0,4,2.266
1,PBKS,Punjab Kings,2,2,0,0,0,4,1.485
2,DC,Delhi Capitals,2,2,0,0,0,4,1.320
3,GT,Gujarat Titans,2,1,1,0,0,2,0.625
4,MI,Mumbai Indians,3,1,2,0,0,2,0.309
5,LSG,Lucknow Super Giants,3,1,2,0,0,2,-0.150
6,CSK,Chennai Super Kings,3,1,2,0,0,2,-0.771
7,SRH,Sunrisers Hyderabad,3,1,2,0,0,2,-0.871
8,RR,Rajasthan Royals,3,1,2,0,0,2,-1.112
9,KKR,Kolkata Knight Riders,3,1,2,0,0,2,-1.428


In [6]:
# let's make it sleep for sometime
time.sleep(random.uniform(2, 5))

## 2: Match Summary

In [7]:
# Step 1: Send a GET request to the website
url = 'https://www.espncricinfo.com/records/season/team-match-results/2025-2025?trophy=117'
response = requests.get(url, headers=headers)

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Step 3: Find the table rows and extract the data from the cells
allRows = soup.select('table > tbody > tr')
matchSummary = []

for element in allRows:
    tds = element.find_all('td')

    if tds[0].text.strip() == 'Punjab Kings':
        team1 = 'PBKS'
    else:
        team1 = tds[0].text.strip()
        
    if tds[1].text.strip() == 'Punjab Kings':
        team2 = 'PBKS'
    else:
        team2 = tds[1].text.strip()

    if tds[2].text.strip() == 'Punjab Kings':
        winner = 'PBKS'
    else:
        winner = tds[2].text.strip()
        
    margin = tds[3].text.strip()
    ground = tds[4].text.strip()
    matchDate = tds[5].text.strip()
    scorecard_link = "https://www.espncricinfo.com" + tds[6].find('a')['href']

    matchSummary.append({
        'team1': team1,
        'team2': team2,
        'winner': winner,
        'margin': margin,
        'ground': ground,
        'matchDate': matchDate,
        'scorecard_link': scorecard_link
    })

# Step 4: Creating a new list to filter out matches with a valid margin
newMatchSummary = []

for match in matchSummary:
    if match['margin'] != '-':
        if ('wickets' in match['margin']) or ('wicket' in match['margin']):
            finT2 = match['winner']
            finT1 = match['team1'] if match['team1'] != match['winner'] else match['team2']
        elif ('runs' in match['margin']) or ('run' in match['margin']):
            finT1 = match['winner']
            finT2 = match['team1'] if match['team1'] != match['winner'] else match['team2']

        newMatchSummary.append({
            'match': f'{finT1} Vs {finT2}',
            'team1': finT1,
            'team2': finT2,
            'winner': match['winner'],
            'margin': match['margin'],
            'matchDate': match['matchDate'],
            'scorecard_link': match['scorecard_link']
        })

# Step 5: Convert the data into a pandas DataFrame
df_match_summary = pd.DataFrame(newMatchSummary)


# Step6: Let's add match ID
df_match_summary['combined_cols'] = df_match_summary['team1'] + ' Vs ' + df_match_summary['team2'] + ' Vs ' + df_match_summary['matchDate']

#create a list of all unique values 
all_matches = df_match_summary['combined_cols'].unique().tolist()

#let assign a unique id for each match_id 
unique_ids = set()
while len(unique_ids) < len(all_matches):
    unique_id = "T20" + str(random.randint(1000, 9999))
    unique_ids.add(unique_id)
unique_ids_list = list(unique_ids)

# create a dictionary for this
dict_matches = {}
for match_combined, match_id in zip(all_matches, unique_ids):
    dict_matches[match_combined] = match_id

# add new column
df_match_summary['matchID'] = df_match_summary['combined_cols'].map(dict_matches)

# drop combined cols
df_match_summary.drop("combined_cols", axis = 1, inplace = True)

# re-arrange cols
cols = ['matchID', 'matchDate', 'match', 'team1', 'team2', 'winner', 'margin', 'scorecard_link']
df_match_summary = df_match_summary[cols]

print(df_match_summary.shape)
df_match_summary.head()

(13, 8)


,matchID,matchDate,match,team1,team2,winner,margin,scorecard_link
0,T202100,"Apr 1, 2025",LSG Vs PBKS,LSG,PBKS,PBKS,8 wickets,https://www.espncricinfo.com/series/ipl-2025-1...
1,T207040,"Mar 31, 2025",KKR Vs MI,KKR,MI,MI,8 wickets,https://www.espncricinfo.com/series/ipl-2025-1...
2,T204583,"Mar 30, 2025",RR Vs CSK,RR,CSK,RR,6 runs,https://www.espncricinfo.com/series/ipl-2025-1...
3,T209474,"Mar 30, 2025",SRH Vs DC,SRH,DC,DC,7 wickets,https://www.espncricinfo.com/series/ipl-2025-1...
4,T201002,"Mar 29, 2025",GT Vs MI,GT,MI,GT,36 runs,https://www.espncricinfo.com/series/ipl-2025-1...


In [8]:
# let's make it sleep for sometime
time.sleep(random.uniform(2, 5))

## 3: Batting | Bowling Summary + Players Links

In [9]:
# define the lists
batting_summary = []
bowling_summary = []
extra_match_info = []

# Step 1: Iterate through each row in the dataframe
for _, row in df_match_summary.iterrows():

    #list to get all team players (To get captain information)
    team1_players = []
    team2_players = []
    
    matchID = row['matchID']
    team1 = row['team1']
    team2 = row['team2']
    match_info = row['match']
    scorecard_link = row['scorecard_link']
    matchDate = row['matchDate']

    #2: Navigate to scorecard link
    response = requests.get(scorecard_link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    #3: Get the batting information tables
    batting_tables = soup.select('div > table.ci-scorecard-table')
    first_inning_batting_rows = [row for row in batting_tables[0].select('tbody > tr') if len(row.select('td')) >= 8]
    second_inning_batting_rows = [row for row in batting_tables[1].select('tbody > tr') if len(row.select('td')) >= 8]


    # 4: Get the bowling information tables
    bowling_tables = soup.select('div > table.ds-table')
    first_inning_bowling_rows = [row for row in bowling_tables[1].select('tbody > tr') if len(row.select('td')) >= 11]
    second_inning_bowling_rows = [row for row in bowling_tables[3].select('tbody > tr') if len(row.select('td')) >= 11]

    # 5: Get the batting records
    #First Innings
    for i in range(0, len(first_inning_batting_rows)):
        element = first_inning_batting_rows[i]
        index = i
    
        tds = element.find_all('td')
        
        #for dismal (handling specially)
        if tds[1].find('span') is not None:
            dismissal =  tds[1].find('span').find('span').text
        else:
            dismissal = tds[1].text.strip()

        batting_summary.append({
            "matchID": matchID,
            "match": match_info,
            "teamInnings": team1,
            "battingPos": index + 1,
            "batsmanName": tds[0].find('a').find_all('span')[1].text.replace('â€', '').replace('\xa0', ''),
            "dismissal": dismissal,
            "runs": tds[2].find('strong').text,
            "balls": tds[3].text,
            "4s": tds[5].text,
            "6s": tds[6].text,
            "SR": tds[7].text,
            "Innings": "1st Innings"
        })
        
        team1_players.append(tds[0].find('a').find_all('span')[1].text.replace('â€', '').replace('\xa0', '')) 
    
    #Second Innings
    for i in range(0, len(second_inning_batting_rows)):
        element = second_inning_batting_rows[i]
        index = i
    
        tds = element.find_all('td')
        
        #for dismal (handling specially)
        if tds[1].find('span') is not None:
            dismissal =  tds[1].find('span').find('span').text
        else:
            dismissal = tds[1].text.strip()

        batting_summary.append({
            "matchID": matchID,
            "match": match_info,
            "teamInnings": team2,
            "battingPos": index + 1,
            "batsmanName": tds[0].find('a').find_all('span')[1].text.replace('â€', '').replace('\xa0', ''),
            "dismissal": dismissal,
            "runs": tds[2].find('strong').text,
            "balls": tds[3].text,
            "4s": tds[5].text,
            "6s": tds[6].text,
            "SR": tds[7].text,
            "Innings": "2nd Innings"
        })

        team2_players.append(tds[0].find('a').find_all('span')[1].text.replace('â€', '').replace('\xa0', ''))



    # did not bat (team1 players)
    try:
        did_not_bat_players = soup.select('div.ds-text-tight-m.ds-font-regular.ds-leading-4.ds-text-typo-mid1')

        if len(did_not_bat_players) == 2:
            # team1
            team1_did_not_bat = did_not_bat_players[0]
            all_spans_team1 = team1_did_not_bat.find_all('span')
            team1_span_texts = list(set([span.get_text(strip=True).replace(',', '') for span in all_spans_team1]))
            team1_players = team1_players + team1_span_texts
    
            # team2
            team2_did_not_bat = did_not_bat_players[1]
            all_spans_team2 = team2_did_not_bat.find_all('span')
            team2_span_texts = list(set([span.get_text(strip=True).replace(',', '') for span in all_spans_team2]))
            team2_players = team2_players + team2_span_texts

        elif len(did_not_bat_players) == 1:
            if len(team1_players) != 11:
                team1_did_not_bat = did_not_bat_players[0]
                all_spans_team1 = team1_did_not_bat.find_all('span')
                team1_span_texts = list(set([span.get_text(strip=True).replace(',', '') for span in all_spans_team1]))
                team1_players = team1_players + team1_span_texts
            else:
                team2_did_not_bat = did_not_bat_players[0]
                all_spans_team2 = team2_did_not_bat.find_all('span')
                team2_span_texts = list(set([span.get_text(strip=True).replace(',', '') for span in all_spans_team2]))
                team2_players = team2_players + team2_span_texts
    except Exception:
        pass  # Ignore all exceptions silently


    # 6: Get the bowling records
    #First Innings
    for index, element in enumerate(first_inning_bowling_rows):
            tds = element.find_all('td')
            bowling_summary.append({
                "matchID": matchID,
                "match": match_info,
                "bowlingTeam": team2,
                "bowlerName": tds[0].find('a').find('span').text.replace('â€', '').replace('\xa0', ''),
                "overs": tds[1].text,
                "maiden": tds[2].text,
                "runs": tds[3].text,
                "wickets": tds[4].text,
                "economy": tds[5].text,
                "0s": tds[6].text,
                "4s": tds[7].text,
                "6s": tds[8].text,
                "wides": tds[9].text,
                "noBalls": tds[10].text,
                "Innings": "1st Innings"
            })


    #Second Innings
    for index, element in enumerate(second_inning_bowling_rows):
        tds = element.find_all('td')
        bowling_summary.append({
            "matchID": matchID,
            "match": match_info,
            "bowlingTeam": team1,
            "bowlerName": tds[0].find('a').find('span').text.replace('â€', '').replace('\xa0', ''),
            "overs": tds[1].text,
            "maiden": tds[2].text,
            "runs": tds[3].text,
            "wickets": tds[4].text,
            "economy": tds[5].text,
            "0s": tds[6].text,
            "4s": tds[7].text,
            "6s": tds[8].text,
            "wides": tds[9].text,
            "noBalls": tds[10].text,
            "Innings": "2nd Innings"
        })


    # 7: Get Extra match info
    team_scores = soup.select('div.ds-mb-1 > div.ci-team-score')
    team1_ex = team_scores[0].select('div')[0].text
    team1_score = team_scores[0].select('div')[1].text
    team2_ex = team_scores[1].select('div')[0].text
    team2_score = team_scores[1].select('div')[1].text
    result_text = soup.select('p.ds-text-tight-s.ds-font-medium.ds-truncate.ds-text-typo')[0].text
    match_location_date = soup.select('div.ds-text-tight-m.ds-font-regular.ds-text-typo-mid3')[0].text
    # toss and player of match
    k_table = soup.select('div.ds-p-0 > table.ds-w-full.ds-table.ds-table-sm.ds-table-auto')[0]
    toss = k_table.select('tr')[1].text.replace('Toss', '')

    if 'Player Of The Match' in k_table.select('tr')[4].text:
        player_of_the_match = k_table.select('tr')[4].text.replace('Player Of The Match', '').replace('â€', '').replace('\xa0', '').replace("(c)", "").replace('†', '')
    else:
        player_of_the_match = k_table.select('tr')[5].text.replace('Player Of The Match', '').replace('â€', '').replace('\xa0', '').replace("(c)", "").replace('†', '')
    

    #team1 captain
    if '(c)' in team1_players:
        team1_players.remove('(c)')
    team1_captain = [player for player in team1_players if "(c)" in player]
    team1_captain_name = team1_captain[0].replace("(c)", "").replace('†', '').strip() if team1_captain else None

    #team2 captain
    if '(c)' in team2_players:
        team2_players.remove('(c)')
    team2_captain = [player for player in team2_players if "(c)" in player]
    team2_captain_name = team2_captain[0].replace("(c)", "").replace('†', '').strip() if team2_captain else None

    extra_match_info.append({
            "matchID": matchID,
            "team1": team1_ex,
            "team1_captain": team1_captain_name,
            "team1_score":  team1_score,
            "team2": team2_ex,
            "team2_captain": team2_captain_name,
            "team2_score": team2_score, 
            "result": result_text,
            "match_location_date": match_location_date,
            "toss": toss,
            "player_of_the_match": player_of_the_match
        })

#creating individual dataframes
batting_summary_df = pd.DataFrame(batting_summary)
bowling_summary_df = pd.DataFrame(bowling_summary)
extra_match_info = pd.DataFrame(extra_match_info)



# data cleaning
#batting
batting_summary_df['batsmanName'] = batting_summary_df['batsmanName'].apply(lambda x: x.replace('(c)', ''))
batting_summary_df['batsmanName']= batting_summary_df['batsmanName'].apply(lambda x: x.replace('†', ''))
batting_summary_df['out/not_out'] = batting_summary_df.dismissal.apply(lambda x: "out" if x != "not out" else "not_out")
batting_summary_df.drop(columns=["dismissal"], inplace=True)

#bowling
bowling_summary_df['bowlerName'] = bowling_summary_df['bowlerName'].apply(lambda x: x.replace('(c)', ''))
bowling_summary_df['bowlerName'] = bowling_summary_df['bowlerName'].apply(lambda x: x.replace('†', ''))


In [10]:
# let's make it sleep for sometime
time.sleep(random.uniform(2, 5))

**Batting**

In [11]:
print(batting_summary_df.shape)
batting_summary_df.head(5)

(217, 12)


,matchID,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Innings,out/not_out
0,T202100,LSG Vs PBKS,LSG,1,Aiden Markram,28,18,4,1,155.55,1st Innings,out
1,T202100,LSG Vs PBKS,LSG,2,Mitchell Marsh,0,1,0,0,0.00,1st Innings,out
2,T202100,LSG Vs PBKS,LSG,3,Nicholas Pooran,44,30,5,2,146.66,1st Innings,out
3,T202100,LSG Vs PBKS,LSG,4,Rishabh Pant,2,5,0,0,40.00,1st Innings,out
4,T202100,LSG Vs PBKS,LSG,5,Ayush Badoni,41,33,1,3,124.24,1st Innings,out


In [12]:
batting_summary_df.tail(5)

,matchID,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Innings,out/not_out
212,T202861,KKR Vs RCB,RCB,1,Phil Salt,56,31,9,2,180.64,2nd Innings,out
213,T202861,KKR Vs RCB,RCB,2,Virat Kohli,59,36,4,3,163.88,2nd Innings,not_out
214,T202861,KKR Vs RCB,RCB,3,Devdutt Padikkal,10,10,1,0,100.00,2nd Innings,out
215,T202861,KKR Vs RCB,RCB,4,Rajat Patidar,34,16,5,1,212.50,2nd Innings,out
216,T202861,KKR Vs RCB,RCB,5,Liam Livingstone,15,5,2,1,300.00,2nd Innings,not_out


**Bowling**

In [13]:
print(bowling_summary_df.shape)
bowling_summary_df.head(6)

(157, 15)


,matchID,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,Innings
0,T202100,LSG Vs PBKS,PBKS,Arshdeep Singh,4,0,43,3,10.75,8,6,1,2,0,1st Innings
1,T202100,LSG Vs PBKS,PBKS,Lockie Ferguson,3,0,26,1,8.66,7,1,2,1,0,1st Innings
2,T202100,LSG Vs PBKS,PBKS,Glenn Maxwell,3,0,22,1,7.33,8,4,0,0,0,1st Innings
3,T202100,LSG Vs PBKS,PBKS,Marco Jansen,4,0,28,1,7.00,9,0,2,1,0,1st Innings
4,T202100,LSG Vs PBKS,PBKS,Marcus Stoinis,2,0,15,0,7.50,5,1,1,0,0,1st Innings
5,T202100,LSG Vs PBKS,PBKS,Yuzvendra Chahal,4,0,36,1,9.00,8,3,2,0,0,1st Innings


In [14]:
bowling_summary_df.tail(6)

,matchID,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,Innings
151,T202861,KKR Vs RCB,RCB,Liam Livingstone,2,0,14,0,7.00,2,1,0,0,0,1st Innings
152,T202861,KKR Vs RCB,KKR,Vaibhav Arora,3,0,42,1,14.00,4,7,1,1,0,2nd Innings
153,T202861,KKR Vs RCB,KKR,Spencer Johnson,2.2,0,31,0,13.28,7,3,3,0,0,2nd Innings
154,T202861,KKR Vs RCB,KKR,Varun Chakravarthy,4,0,43,1,10.75,7,5,2,0,0,2nd Innings
155,T202861,KKR Vs RCB,KKR,Harshit Rana,3,0,32,0,10.66,4,5,0,0,0,2nd Innings
156,T202861,KKR Vs RCB,KKR,Sunil Narine,4,0,27,1,6.75,5,1,1,0,0,2nd Innings


**Extra Match Info**

In [15]:
print(extra_match_info.shape)
extra_match_info.head()

(13, 11)


,matchID,team1,team1_captain,team1_score,team2,team2_captain,team2_score,result,match_location_date,toss,player_of_the_match
0,T202100,Lucknow Super Giants,Rishabh Pant,171/7,Punjab Kings,Shreyas Iyer,"(16.2/20 ov, T:172) 177/2",PBKS won by 8 wickets (with 22 balls remaining),"13th Match (N), Lucknow, April 01, 2025, India...","Punjab Kings, elected to field first",Prabhsimran Singh
1,T207040,Kolkata Knight Riders,Ajinkya Rahane,116,Mumbai Indians,Hardik Pandya,"(12.5/20 ov, T:117) 121/2",MI won by 8 wickets (with 43 balls remaining),"12th Match (N), Wankhede, March 31, 2025, Indi...","Mumbai Indians, elected to field first",Ashwani Kumar
2,T204583,Rajasthan Royals,Riyan Parag,182/9,Chennai Super Kings,Ruturaj Gaikwad,"(20 ov, T:183) 176/6",RR won by 6 runs,"11th Match (N), Guwahati, March 30, 2025, Indi...","Chennai Super Kings, elected to field first",Nitish Rana
3,T209474,Sunrisers Hyderabad,Pat Cummins,163,Delhi Capitals,Axar Patel,"(16/20 ov, T:164) 166/3",DC won by 7 wickets (with 24 balls remaining),"10th Match (D/N), Visakhapatnam, March 30, 202...","Sunrisers Hyderabad, elected to bat first",Mitchell Starc
4,T201002,Gujarat Titans,Shubman Gill,196/8,Mumbai Indians,Hardik Pandya,"(20 ov, T:197) 160/6",GT won by 36 runs,"9th Match (N), Ahmedabad, March 29, 2025, Indi...","Mumbai Indians, elected to field first",Prasidh Krishna


In [16]:
extra_match_info.tail()

,matchID,team1,team1_captain,team1_score,team2,team2_captain,team2_score,result,match_location_date,toss,player_of_the_match
8,T209622,Punjab Kings,Shreyas Iyer,243/5,Gujarat Titans,Shubman Gill,"(20 ov, T:244) 232/5",PBKS won by 11 runs,"5th Match (N), Ahmedabad, March 25, 2025, Indi...","Gujarat Titans, elected to field first",Shreyas Iyer
9,T208377,Lucknow Super Giants,Rishabh Pant,209/8,Delhi Capitals,Axar Patel,"(19.3/20 ov, T:210) 211/9",DC won by 1 wicket (with 3 balls remaining),"4th Match (N), Visakhapatnam, March 24, 2025, ...","Delhi Capitals, elected to field first",Ashutosh Sharma
10,T202965,Mumbai Indians,Suryakumar Yadav,155/9,Chennai Super Kings,Ruturaj Gaikwad,"(19.1/20 ov, T:156) 158/6",CSK won by 4 wickets (with 5 balls remaining),"3rd Match (N), Chennai, March 23, 2025, Indian...","Chennai Super Kings, elected to field first",Noor Ahmad
11,T209594,Sunrisers Hyderabad,Pat Cummins,286/6,Rajasthan Royals,Riyan Parag,"(20 ov, T:287) 242/6",SRH won by 44 runs,"2nd Match (D/N), Hyderabad, March 23, 2025, In...","Rajasthan Royals, elected to field first",Ishan Kishan
12,T202861,Kolkata Knight Riders,Ajinkya Rahane,174/8,Royal Challengers Bengaluru,Rajat Patidar,"(16.2/20 ov, T:175) 177/3",RCB won by 7 wickets (with 22 balls remaining),"1st Match (N), Eden Gardens, March 22, 2025, I...","Royal Challengers Bengaluru, elected to field ...",Krunal Pandya


## 4: Processing Match and Extra Match info

In [17]:
print(extra_match_info.shape)
extra_match_info.head()

(13, 11)


,matchID,team1,team1_captain,team1_score,team2,team2_captain,team2_score,result,match_location_date,toss,player_of_the_match
0,T202100,Lucknow Super Giants,Rishabh Pant,171/7,Punjab Kings,Shreyas Iyer,"(16.2/20 ov, T:172) 177/2",PBKS won by 8 wickets (with 22 balls remaining),"13th Match (N), Lucknow, April 01, 2025, India...","Punjab Kings, elected to field first",Prabhsimran Singh
1,T207040,Kolkata Knight Riders,Ajinkya Rahane,116,Mumbai Indians,Hardik Pandya,"(12.5/20 ov, T:117) 121/2",MI won by 8 wickets (with 43 balls remaining),"12th Match (N), Wankhede, March 31, 2025, Indi...","Mumbai Indians, elected to field first",Ashwani Kumar
2,T204583,Rajasthan Royals,Riyan Parag,182/9,Chennai Super Kings,Ruturaj Gaikwad,"(20 ov, T:183) 176/6",RR won by 6 runs,"11th Match (N), Guwahati, March 30, 2025, Indi...","Chennai Super Kings, elected to field first",Nitish Rana
3,T209474,Sunrisers Hyderabad,Pat Cummins,163,Delhi Capitals,Axar Patel,"(16/20 ov, T:164) 166/3",DC won by 7 wickets (with 24 balls remaining),"10th Match (D/N), Visakhapatnam, March 30, 202...","Sunrisers Hyderabad, elected to bat first",Mitchell Starc
4,T201002,Gujarat Titans,Shubman Gill,196/8,Mumbai Indians,Hardik Pandya,"(20 ov, T:197) 160/6",GT won by 36 runs,"9th Match (N), Ahmedabad, March 29, 2025, Indi...","Mumbai Indians, elected to field first",Prasidh Krishna


In [18]:
print(df_match_summary.shape)
df_match_summary.head()

(13, 8)


,matchID,matchDate,match,team1,team2,winner,margin,scorecard_link
0,T202100,"Apr 1, 2025",LSG Vs PBKS,LSG,PBKS,PBKS,8 wickets,https://www.espncricinfo.com/series/ipl-2025-1...
1,T207040,"Mar 31, 2025",KKR Vs MI,KKR,MI,MI,8 wickets,https://www.espncricinfo.com/series/ipl-2025-1...
2,T204583,"Mar 30, 2025",RR Vs CSK,RR,CSK,RR,6 runs,https://www.espncricinfo.com/series/ipl-2025-1...
3,T209474,"Mar 30, 2025",SRH Vs DC,SRH,DC,DC,7 wickets,https://www.espncricinfo.com/series/ipl-2025-1...
4,T201002,"Mar 29, 2025",GT Vs MI,GT,MI,GT,36 runs,https://www.espncricinfo.com/series/ipl-2025-1...


In [19]:
### Merge both the columns

df_final_match_summary = df_match_summary.merge(extra_match_info, on = 'matchID')

req_cols = ['matchID', 'matchDate', 'match', 'match_location_date', 'toss', 'team1_x', 'team1_captain', 'team1_score', 
            'team2_x', 'team2_captain', 'team2_score', 'winner', 'result', 'player_of_the_match']

df_final_match_summary = df_final_match_summary[req_cols]

#Rename cols
rename_cols = {'team1_x': 'team1', 'team2_x': 'team2'}
df_final_match_summary.rename(rename_cols, axis = 1, inplace = True)

print(df_final_match_summary.shape)
df_final_match_summary.head()

(13, 14)


,matchID,matchDate,match,match_location_date,toss,team1,team1_captain,team1_score,team2,team2_captain,team2_score,winner,result,player_of_the_match
0,T202100,"Apr 1, 2025",LSG Vs PBKS,"13th Match (N), Lucknow, April 01, 2025, India...","Punjab Kings, elected to field first",LSG,Rishabh Pant,171/7,PBKS,Shreyas Iyer,"(16.2/20 ov, T:172) 177/2",PBKS,PBKS won by 8 wickets (with 22 balls remaining),Prabhsimran Singh
1,T207040,"Mar 31, 2025",KKR Vs MI,"12th Match (N), Wankhede, March 31, 2025, Indi...","Mumbai Indians, elected to field first",KKR,Ajinkya Rahane,116,MI,Hardik Pandya,"(12.5/20 ov, T:117) 121/2",MI,MI won by 8 wickets (with 43 balls remaining),Ashwani Kumar
2,T204583,"Mar 30, 2025",RR Vs CSK,"11th Match (N), Guwahati, March 30, 2025, Indi...","Chennai Super Kings, elected to field first",RR,Riyan Parag,182/9,CSK,Ruturaj Gaikwad,"(20 ov, T:183) 176/6",RR,RR won by 6 runs,Nitish Rana
3,T209474,"Mar 30, 2025",SRH Vs DC,"10th Match (D/N), Visakhapatnam, March 30, 202...","Sunrisers Hyderabad, elected to bat first",SRH,Pat Cummins,163,DC,Axar Patel,"(16/20 ov, T:164) 166/3",DC,DC won by 7 wickets (with 24 balls remaining),Mitchell Starc
4,T201002,"Mar 29, 2025",GT Vs MI,"9th Match (N), Ahmedabad, March 29, 2025, Indi...","Mumbai Indians, elected to field first",GT,Shubman Gill,196/8,MI,Hardik Pandya,"(20 ov, T:197) 160/6",GT,GT won by 36 runs,Prasidh Krishna


## 5: Players Data

In [20]:
players_info_df_final = pd.read_csv('abfss://1e8640c6-dc84-47a8-b295-452f8a310af8@onelake.dfs.fabric.microsoft.com/327dd03d-fc61-4a7c-8838-35fa63ac0dd5/Files/2025/dim_players.csv')
print(players_info_df_final.shape)
players_info_df_final.head()

(227, 7)


,name,team,link,battingStyle,bowlingStyle,playingRole,image
0,Heinrich Klaasen,SRH,https://www.espncricinfo.com//cricketers/heinr...,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...
1,Pat Cummins,SRH,https://www.espncricinfo.com//cricketers/pat-c...,Right hand Bat,Right arm Fast,Bowler,https://documents.iplt20.com/ipl/IPLHeadshot20...
2,Abhishek Sharma,SRH,https://www.espncricinfo.com//cricketers/abhis...,Left hand Bat,Slow Left arm Orthodox,Batting Allrounder,https://documents.iplt20.com/ipl/IPLHeadshot20...
3,Travis Head,SRH,https://www.espncricinfo.com//cricketers/travi...,Left hand Bat,Right arm Offbreak,Top order Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...
4,Ishan Kishan,SRH,https://www.espncricinfo.com//cricketers/ishan...,Left hand Bat,Legbreak,Wicketkeeper Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...


**Comparison Check**

In [21]:
all_batsman_names = batting_summary_df['batsmanName'].unique().tolist()
all_bowler_names = bowling_summary_df['bowlerName'].unique().tolist()
all_player_names = players_info_df_final['name'].unique().tolist()

print('Total Players in Batsman: ', len(all_batsman_names))
print('Total Players in Bowlers: ', len(all_bowler_names))
print('Total Players: ', len(all_player_names))

Total Players in Batsman:  111
Total Players in Bowlers:  80
Total Players:  227


In [22]:
#Batsman Check: (If null, then all batsman players present in dim_players)

for batsman_play in all_batsman_names:
    if batsman_play not in all_player_names:
        print(batsman_play)

In [23]:
#Bowlers Check: (If null, then all bowler players present in dim_players)

for bowl_play in all_bowler_names:
    if bowl_play not in all_player_names:
        print(bowl_play)

### Adding players images in match summary

In [24]:
print(df_final_match_summary.shape)
df_final_match_summary.head()

(13, 14)


,matchID,matchDate,match,match_location_date,toss,team1,team1_captain,team1_score,team2,team2_captain,team2_score,winner,result,player_of_the_match
0,T202100,"Apr 1, 2025",LSG Vs PBKS,"13th Match (N), Lucknow, April 01, 2025, India...","Punjab Kings, elected to field first",LSG,Rishabh Pant,171/7,PBKS,Shreyas Iyer,"(16.2/20 ov, T:172) 177/2",PBKS,PBKS won by 8 wickets (with 22 balls remaining),Prabhsimran Singh
1,T207040,"Mar 31, 2025",KKR Vs MI,"12th Match (N), Wankhede, March 31, 2025, Indi...","Mumbai Indians, elected to field first",KKR,Ajinkya Rahane,116,MI,Hardik Pandya,"(12.5/20 ov, T:117) 121/2",MI,MI won by 8 wickets (with 43 balls remaining),Ashwani Kumar
2,T204583,"Mar 30, 2025",RR Vs CSK,"11th Match (N), Guwahati, March 30, 2025, Indi...","Chennai Super Kings, elected to field first",RR,Riyan Parag,182/9,CSK,Ruturaj Gaikwad,"(20 ov, T:183) 176/6",RR,RR won by 6 runs,Nitish Rana
3,T209474,"Mar 30, 2025",SRH Vs DC,"10th Match (D/N), Visakhapatnam, March 30, 202...","Sunrisers Hyderabad, elected to bat first",SRH,Pat Cummins,163,DC,Axar Patel,"(16/20 ov, T:164) 166/3",DC,DC won by 7 wickets (with 24 balls remaining),Mitchell Starc
4,T201002,"Mar 29, 2025",GT Vs MI,"9th Match (N), Ahmedabad, March 29, 2025, Indi...","Mumbai Indians, elected to field first",GT,Shubman Gill,196/8,MI,Hardik Pandya,"(20 ov, T:197) 160/6",GT,GT won by 36 runs,Prasidh Krishna


In [25]:
# new cols to add
# 1. team1_logo, team2_logo, team1_captain_logo, team2_captain_logo, player_of_match_logo


# step1: Add team1 and team2 logo and also full form of team
ipl_teams_logos = {
    'KKR': 'https://documents.iplt20.com/ipl/KKR/Logos/Logooutline/KKRoutline.png',
    'SRH': 'https://documents.iplt20.com/ipl/SRH/Logos/Logooutline/SRHoutline.png',
    'RR': 'https://documents.iplt20.com/ipl/RR/Logos/Logooutline/RRoutline.png',
    'RCB': 'https://documents.iplt20.com/ipl/RCB/Logos/Logooutline/RCBoutline.png',
    'CSK': 'https://documents.iplt20.com/ipl/CSK/logos/Logooutline/CSKoutline.png',
    'DC': 'https://documents.iplt20.com/ipl/DC/Logos/LogoOutline/DCoutline.png',
    'LSG': 'https://documents.iplt20.com/ipl/LSG/Logos/Logooutline/LSGoutline.png',
    'GT': 'https://documents.iplt20.com/ipl/GT/Logos/Logooutline/GToutline.png',
    'PBKS': 'https://documents.iplt20.com/ipl/PBKS/Logos/Logooutline/PBKSoutline.png', 
    'MI': 'https://documents.iplt20.com/ipl/MI/Logos/Logooutline/MIoutline.png'
}

ipl_teams_names = {
    "CSK": "Chennai Super Kings",
    "DC": "Delhi Capitals",
    "GT": "Gujarat Titans",
    "KKR": "Kolkata Knight Riders",
    "LSG": "Lucknow Super Giants",
    "MI": "Mumbai Indians",
    "PBKS": "Punjab Kings",
    "RR": "Rajasthan Royals",
    "RCB": "Royal Challengers Bengaluru",
    "SRH": "Sunrisers Hyderabad"
}

#logo
df_final_match_summary['team1_logo'] = df_final_match_summary['team1'].map(ipl_teams_logos)
df_final_match_summary['team2_logo'] = df_final_match_summary['team2'].map(ipl_teams_logos)

# full form
df_final_match_summary['team1_full_name'] = df_final_match_summary['team1'].map(ipl_teams_names)
df_final_match_summary['team2_full_name'] = df_final_match_summary['team2'].map(ipl_teams_names)


# step2: Add players images (team1 captain, team2 captain, player_of_match_img
# players dictionary
name_image_dict = players_info_df_final.set_index("name")["image"].to_dict()

df_final_match_summary['team1_captain_image'] = df_final_match_summary['team1_captain'].map(name_image_dict)
df_final_match_summary['team2_captain_image'] = df_final_match_summary['team2_captain'].map(name_image_dict)
df_final_match_summary['player_of_the_match_image'] = df_final_match_summary['player_of_the_match'].map(name_image_dict)

In [26]:
print(df_final_match_summary.shape)
df_final_match_summary.head()

(13, 21)


,matchID,matchDate,match,match_location_date,toss,team1,team1_captain,team1_score,team2,team2_captain,...,winner,result,player_of_the_match,team1_logo,team2_logo,team1_full_name,team2_full_name,team1_captain_image,team2_captain_image,player_of_the_match_image
0,T202100,"Apr 1, 2025",LSG Vs PBKS,"13th Match (N), Lucknow, April 01, 2025, India...","Punjab Kings, elected to field first",LSG,Rishabh Pant,171/7,PBKS,Shreyas Iyer,...,PBKS,PBKS won by 8 wickets (with 22 balls remaining),Prabhsimran Singh,https://documents.iplt20.com/ipl/LSG/Logos/Log...,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...,Lucknow Super Giants,Punjab Kings,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
1,T207040,"Mar 31, 2025",KKR Vs MI,"12th Match (N), Wankhede, March 31, 2025, Indi...","Mumbai Indians, elected to field first",KKR,Ajinkya Rahane,116,MI,Hardik Pandya,...,MI,MI won by 8 wickets (with 43 balls remaining),Ashwani Kumar,https://documents.iplt20.com/ipl/KKR/Logos/Log...,https://documents.iplt20.com/ipl/MI/Logos/Logo...,Kolkata Knight Riders,Mumbai Indians,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
2,T204583,"Mar 30, 2025",RR Vs CSK,"11th Match (N), Guwahati, March 30, 2025, Indi...","Chennai Super Kings, elected to field first",RR,Riyan Parag,182/9,CSK,Ruturaj Gaikwad,...,RR,RR won by 6 runs,Nitish Rana,https://documents.iplt20.com/ipl/RR/Logos/Logo...,https://documents.iplt20.com/ipl/CSK/logos/Log...,Rajasthan Royals,Chennai Super Kings,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
3,T209474,"Mar 30, 2025",SRH Vs DC,"10th Match (D/N), Visakhapatnam, March 30, 202...","Sunrisers Hyderabad, elected to bat first",SRH,Pat Cummins,163,DC,Axar Patel,...,DC,DC won by 7 wickets (with 24 balls remaining),Mitchell Starc,https://documents.iplt20.com/ipl/SRH/Logos/Log...,https://documents.iplt20.com/ipl/DC/Logos/Logo...,Sunrisers Hyderabad,Delhi Capitals,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
4,T201002,"Mar 29, 2025",GT Vs MI,"9th Match (N), Ahmedabad, March 29, 2025, Indi...","Mumbai Indians, elected to field first",GT,Shubman Gill,196/8,MI,Hardik Pandya,...,GT,GT won by 36 runs,Prasidh Krishna,https://documents.iplt20.com/ipl/GT/Logos/Logo...,https://documents.iplt20.com/ipl/MI/Logos/Logo...,Gujarat Titans,Mumbai Indians,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...


## Data Preview

In [27]:
# teams data

print(df_ipl_teams.shape)
df_ipl_teams.head(10)

(10, 3)


,team_code,team,team_logo
0,CSK,Chennai Super Kings,https://documents.iplt20.com/ipl/CSK/logos/Log...
1,DC,Delhi Capitals,https://documents.iplt20.com/ipl/DC/Logos/Logo...
2,GT,Gujarat Titans,https://documents.iplt20.com/ipl/GT/Logos/Logo...
3,KKR,Kolkata Knight Riders,https://documents.iplt20.com/ipl/KKR/Logos/Log...
4,LSG,Lucknow Super Giants,https://documents.iplt20.com/ipl/LSG/Logos/Log...
5,MI,Mumbai Indians,https://documents.iplt20.com/ipl/MI/Logos/Logo...
6,PBKS,Punjab Kings,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...
7,RR,Rajasthan Royals,https://documents.iplt20.com/ipl/RR/Logos/Logo...
8,RCB,Royal Challengers Bengaluru,https://documents.iplt20.com/ipl/RCB/Logos/Log...
9,SRH,Sunrisers Hyderabad,https://documents.iplt20.com/ipl/SRH/Logos/Log...


In [28]:
# Points Table
print(df_points_table.shape)
df_points_table.head()

(10, 9)


,team_code,team,matches_played,won,lost,tied,abonded,points,net_run_rate
0,RCB,Royal Challengers Bengaluru,2,2,0,0,0,4,2.266
1,PBKS,Punjab Kings,2,2,0,0,0,4,1.485
2,DC,Delhi Capitals,2,2,0,0,0,4,1.320
3,GT,Gujarat Titans,2,1,1,0,0,2,0.625
4,MI,Mumbai Indians,3,1,2,0,0,2,0.309


In [29]:
# Match Summary
print(df_final_match_summary.shape)
df_final_match_summary.head()

(13, 21)


,matchID,matchDate,match,match_location_date,toss,team1,team1_captain,team1_score,team2,team2_captain,...,winner,result,player_of_the_match,team1_logo,team2_logo,team1_full_name,team2_full_name,team1_captain_image,team2_captain_image,player_of_the_match_image
0,T202100,"Apr 1, 2025",LSG Vs PBKS,"13th Match (N), Lucknow, April 01, 2025, India...","Punjab Kings, elected to field first",LSG,Rishabh Pant,171/7,PBKS,Shreyas Iyer,...,PBKS,PBKS won by 8 wickets (with 22 balls remaining),Prabhsimran Singh,https://documents.iplt20.com/ipl/LSG/Logos/Log...,https://documents.iplt20.com/ipl/PBKS/Logos/Lo...,Lucknow Super Giants,Punjab Kings,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
1,T207040,"Mar 31, 2025",KKR Vs MI,"12th Match (N), Wankhede, March 31, 2025, Indi...","Mumbai Indians, elected to field first",KKR,Ajinkya Rahane,116,MI,Hardik Pandya,...,MI,MI won by 8 wickets (with 43 balls remaining),Ashwani Kumar,https://documents.iplt20.com/ipl/KKR/Logos/Log...,https://documents.iplt20.com/ipl/MI/Logos/Logo...,Kolkata Knight Riders,Mumbai Indians,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
2,T204583,"Mar 30, 2025",RR Vs CSK,"11th Match (N), Guwahati, March 30, 2025, Indi...","Chennai Super Kings, elected to field first",RR,Riyan Parag,182/9,CSK,Ruturaj Gaikwad,...,RR,RR won by 6 runs,Nitish Rana,https://documents.iplt20.com/ipl/RR/Logos/Logo...,https://documents.iplt20.com/ipl/CSK/logos/Log...,Rajasthan Royals,Chennai Super Kings,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
3,T209474,"Mar 30, 2025",SRH Vs DC,"10th Match (D/N), Visakhapatnam, March 30, 202...","Sunrisers Hyderabad, elected to bat first",SRH,Pat Cummins,163,DC,Axar Patel,...,DC,DC won by 7 wickets (with 24 balls remaining),Mitchell Starc,https://documents.iplt20.com/ipl/SRH/Logos/Log...,https://documents.iplt20.com/ipl/DC/Logos/Logo...,Sunrisers Hyderabad,Delhi Capitals,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...
4,T201002,"Mar 29, 2025",GT Vs MI,"9th Match (N), Ahmedabad, March 29, 2025, Indi...","Mumbai Indians, elected to field first",GT,Shubman Gill,196/8,MI,Hardik Pandya,...,GT,GT won by 36 runs,Prasidh Krishna,https://documents.iplt20.com/ipl/GT/Logos/Logo...,https://documents.iplt20.com/ipl/MI/Logos/Logo...,Gujarat Titans,Mumbai Indians,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...,https://documents.iplt20.com/ipl/IPLHeadshot20...


In [30]:
# batting

print(batting_summary_df.shape)
batting_summary_df.head()

(217, 12)


,matchID,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,Innings,out/not_out
0,T202100,LSG Vs PBKS,LSG,1,Aiden Markram,28,18,4,1,155.55,1st Innings,out
1,T202100,LSG Vs PBKS,LSG,2,Mitchell Marsh,0,1,0,0,0.00,1st Innings,out
2,T202100,LSG Vs PBKS,LSG,3,Nicholas Pooran,44,30,5,2,146.66,1st Innings,out
3,T202100,LSG Vs PBKS,LSG,4,Rishabh Pant,2,5,0,0,40.00,1st Innings,out
4,T202100,LSG Vs PBKS,LSG,5,Ayush Badoni,41,33,1,3,124.24,1st Innings,out


In [31]:
# bowling

print(bowling_summary_df.shape)
bowling_summary_df.head()

(157, 15)


,matchID,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,Innings
0,T202100,LSG Vs PBKS,PBKS,Arshdeep Singh,4,0,43,3,10.75,8,6,1,2,0,1st Innings
1,T202100,LSG Vs PBKS,PBKS,Lockie Ferguson,3,0,26,1,8.66,7,1,2,1,0,1st Innings
2,T202100,LSG Vs PBKS,PBKS,Glenn Maxwell,3,0,22,1,7.33,8,4,0,0,0,1st Innings
3,T202100,LSG Vs PBKS,PBKS,Marco Jansen,4,0,28,1,7.00,9,0,2,1,0,1st Innings
4,T202100,LSG Vs PBKS,PBKS,Marcus Stoinis,2,0,15,0,7.50,5,1,1,0,0,1st Innings


In [32]:
# players data

print(players_info_df_final.shape)
players_info_df_final.head()

(227, 7)


,name,team,link,battingStyle,bowlingStyle,playingRole,image
0,Heinrich Klaasen,SRH,https://www.espncricinfo.com//cricketers/heinr...,Right hand Bat,Right arm Offbreak,Wicketkeeper Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...
1,Pat Cummins,SRH,https://www.espncricinfo.com//cricketers/pat-c...,Right hand Bat,Right arm Fast,Bowler,https://documents.iplt20.com/ipl/IPLHeadshot20...
2,Abhishek Sharma,SRH,https://www.espncricinfo.com//cricketers/abhis...,Left hand Bat,Slow Left arm Orthodox,Batting Allrounder,https://documents.iplt20.com/ipl/IPLHeadshot20...
3,Travis Head,SRH,https://www.espncricinfo.com//cricketers/travi...,Left hand Bat,Right arm Offbreak,Top order Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...
4,Ishan Kishan,SRH,https://www.espncricinfo.com//cricketers/ishan...,Left hand Bat,Legbreak,Wicketkeeper Batter,https://documents.iplt20.com/ipl/IPLHeadshot20...


### Save all Files

In [33]:
folder = 'abfss://1e8640c6-dc84-47a8-b295-452f8a310af8@onelake.dfs.fabric.microsoft.com/327dd03d-fc61-4a7c-8838-35fa63ac0dd5/Files/2025/'

# # 1: Teams Data (dim_teams.csv)
# df_ipl_teams.to_csv(folder + 'dim_teams.csv', index=False)


# 2: Points Table (dim_points.csv)
df_points_table.to_csv(folder + 'dim_points_table.csv', index = False)


# 3: Match Summary (dim_match_summary.csv)
df_final_match_summary.to_csv(folder + 'dim_match_summary.csv', index = False)


# 4: Batting Summary (fact_batting_summary.csv)
batting_summary_df.to_csv(folder + 'fact_batting_summary.csv', index = False)


# 5: Bowling Summary (fact_bowling_summary.csv)
bowling_summary_df.to_csv(folder + 'fact_bowling_summary.csv', index = False)


# # 6: Players Data (dim_players.csv)
# players_info_df_final.to_csv(folder + 'dim_players.csv', index = False)